In [1]:
from typing import *

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

import os

from lib.dnn.utils import KerasTrainer
from core.utils.cached_optimizer import CachedOptimizer
from lib.dnn.layers import Delta, MovingAverage, Percentage


2022-02-21 23:05:19.402560: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-21 23:05:19.402584: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class Trial0Trainer(KerasTrainer):
    def __init__(self, seq_len, hidden_layers: List[int], *args,
                 loss="binary_crossentropy",
                 optimizer="adam",
                 hidden_activation="relu",
                 delta=True,
                 average_window=None,
                 percentage=True,
                 **kwargs):
        super(Trial0Trainer, self).__init__(*args, **kwargs)
        self.__seq_len = seq_len
        self.__hidden_layers = hidden_layers
        self.__loss = loss
        self.__optimizer = optimizer
        self.__hidden_activation = hidden_activation
        self.__delta = delta
        self.__percentage = percentage
        self.__average_window = average_window
        if average_window is not None:
            self.__average_window = int(self.__seq_len * average_window)

    def __get_currency_pairs(self, df: pd.DataFrame) -> List[Tuple[str, str]]:
        DELIMITER = "/"
        return [(pair.split(DELIMITER)[0], pair.split(DELIMITER)[1]) for pair in
                set(df["base_currency"] + DELIMITER + df["quote_currency"])]  # TODO FIND A CLEAR WAY

    def __prepare_for_pair(self, sequence: np.ndarray, seq_len: int):
        data_len = sequence.shape[0] - seq_len
        X = np.zeros((data_len, seq_len))
        y = np.zeros((data_len,))
        for i in range(data_len):
            X[i] = sequence[i:i + seq_len]
            if sequence[i] > sequence[i - 1]:
                y[i] = 1
            else:
                y[i] = 0
        return X, y

    def _prepare_data(self, data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        currency_pairs = self.__get_currency_pairs(data)

        X = []
        y = []

        for base_currency, quote_currency in currency_pairs:
            pair_sequence = data[data["base_currency"] == base_currency][
                data[data["base_currency"] == base_currency]["quote_currency"] == quote_currency][
                "c"].to_numpy()  # TODO
            pair_X, pair_y = self.__prepare_for_pair(pair_sequence, self.__seq_len)
            X += list(pair_X)
            y += list(pair_y)

        return np.array(X), np.array(y)

    def _create_model(self) -> keras.Model:
        input_layer = keras.layers.Input(shape=self.__seq_len)

        pre_hidden_layer = input_layer

        if self.__average_window != 0:
            pre_hidden_layer = MovingAverage(self.__average_window)(pre_hidden_layer)

        pre_hidden_layer_copy = pre_hidden_layer[:, : -1]

        if self.__delta:
            pre_hidden_layer = Delta()(pre_hidden_layer)
        else:
            pre_hidden_layer = pre_hidden_layer[:, 1:]

        if self.__percentage:
            pre_hidden_layer = tf.concat(
                (
                    tf.reshape(pre_hidden_layer, (-1, 1, pre_hidden_layer.shape[1])),
                    tf.reshape(pre_hidden_layer_copy, (-1, 1, pre_hidden_layer.shape[1]))
                ),
                axis=1
            )
            pre_hidden_layer = Percentage()(pre_hidden_layer)

        output_layer = keras.layers.Dense(1, activation="sigmoid")

        if len(self.__hidden_layers) == 0:
            output_layer = output_layer(pre_hidden_layer)
        else:
            hidden_layer = keras.layers.Dense(self.__hidden_layers[0], activation=self.__hidden_activation)(
                pre_hidden_layer)
            for layer_size in self.__hidden_layers[1:]:
                hidden_layer = keras.layers.Dense(layer_size, activation=self.__hidden_activation)(hidden_layer)
            output_layer = output_layer(hidden_layer)

        model = keras.Model(inputs=input_layer, outputs=output_layer)

        return model

    def _compile_model(self, model: keras.Model):
        model.compile(loss=self.__loss, optimizer=self.__optimizer, metrics=["accuracy"])


In [3]:
from typing import List, Dict


class Trial0Optimizer(CachedOptimizer):

    def __init__(self, user):
        self.__samples = None
        super(Trial0Optimizer, self).__init__(user)
        
    def __load_samples(self):
        return eval(open("temp/notebooks/samples.txt").read())
    
    def _get_samples(self) -> List[Dict]:
        if self.__samples is None:
            self.__samples = self.__load_samples()
        return self.__samples

    # def __generate_possible_layers(self, layer_sizes, depth):
    #     if depth == 1:
    #         return [[size] for size in layer_sizes]
	# 
    #     layers = []
    #     returned_layers = self.__generate_possible_layers(layer_sizes, depth - 1)
    #     layers += returned_layers
    #     for size in layer_sizes:
    #         for layer in returned_layers:
    #             layers += [[size] + layer]
	# 
    #     return layers
	# 
    # def __generate_hidden_layers(self, base, min_power, max_power, max_depth) -> List[List[int]]:
    #     print("[+]Generating Hidden Layer Sizes...")
	# 
    #     layer_sizes = [base ** i for i in range(min_power, max_power)]
	# 
    #     return self.__generate_possible_layers(layer_sizes, max_depth)
	# 
    # def _generate_param_values(self) -> Dict:
    #     return {
    #         "seq_len": [2 ** (i) for i in range(2, 6)],
    #         "hidden_layers": self.__generate_hidden_layers(self.__base, self.__min_power, self.__max_power,
    #                                                        self.__max_depth),
    #         "loss": ["binary_crossentropy"],
    #         "optimizer": ["adam"],
    #         "hidden_activation": ["relu"],
    #         "delta": [True, False],
    #         "percentage": [True, False],
    #         "average_window": [0],
    #         
    #     }

    def _create_trainer(self, params) -> KerasTrainer:
        print("[+]Creating Trainer...")
        return Trial0Trainer(**params, export_path="temp/dummyModel.h5", data_path="Data/Minutes/AUD-CAD.csv", epochs=2)


In [4]:
MAX_POWER = 6
MIN_POWER = 3
BASE = 4


In [5]:
trainer = Trial0Trainer(10, [2, 3], data_path="Data/Minutes/AUD-CAD.csv", export_path="Model.h5", epochs=1, delta=False, average_window=0.4, percentage=False)

In [6]:
#trainer.start()

In [7]:
optimizer = Trial0Optimizer("test_user")


In [8]:
optimizer.optimize()

[+]Creating Trainer...
[+]Starting Training...
[+]Loading Data: Data/Minutes/AUD-CAD.csv


[+]Splitting Data...


2022-02-21 23:05:47.794056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-21 23:05:47.794157: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-21 23:05:47.794369: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dhcppc7): /proc/driver/nvidia/version does not exist
2022-02-21 23:05:47.851067: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[+]Model Summary
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
delta (Delta)                   (None, 3)            0           input_1[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 3)            0           input_1[0][0]                    
__________________________________________________________________________________________________
tf.reshape (TFOpLambda)         (None, 1, 3)         0           delta[0][0]                      
_____________________________________________________________________________

2022-02-21 23:05:49.232588: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-21 23:05:49.291566: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 1999965000 Hz


...

 8782/16150 [===============>..............] - ETA: 21s - loss: 0.6918 - accuracy: 0.5265

 8796/16150 [===============>..............] - ETA: 21s - loss: 0.6918 - accuracy: 0.5265

 8809/16150 [===============>..............] - ETA: 21s - loss: 0.6918 - accuracy: 0.5265

 8826/16150 [===============>..............] - ETA: 21s - loss: 0.6918 - accuracy: 0.5265

KeyboardInterrupt: 

In [9]:
model